In [39]:
from uuid import uuid4

class Node:
    def __init__(self, data, level, fval):
        """ Initialize the node with the data, level of the node and the calculated fvalue """
        self.data = data
        self.level = level
        self.fval = fval

    def generate_child(self, prev_move_direction: str, prev_node_name: str, h_type: str = "h1"):
        # Generate child nodes from the given node by moving the blank space either in the four directions {left,up,right,down}
        # val_list contains position values for moving the blank space in either of the 4 directions [left,up,right,down] respectively.
        x, y = _find(self.data, '_')
        direction_map = {
            "1_left": [x, y - 1],
            "2_up": [x - 1, y],
            "3_right": [x, y + 1],
            "4_down": [x + 1, y]
        }
        opposite_direction_map = {
            "1_left": "3_right",
            "2_up": "4_down",
            "3_right": "1_left",
            "4_down": "2_up",
        }
        child_list = []
        for k,v in direction_map.items():
            if opposite_direction_map[k] != prev_move_direction:
                child = self.shuffle(self.data, x, y, v[0], v[1])
                if child is not None:
                    child_node = Node(child,self.level+1,0)
                    child_list.append(
                        {
                            "node": child_node,
                            "node_name": str(uuid4()),
                            "f": _get_f(child_node, type_=h_type),
                            "prev_node_name": prev_node_name,
                            "prev_move_direction": k
                         }
                    )
        return child_list
        
    def shuffle(self,puz,x1,y1,x2,y2):
        """ Move the blank space in the given direction and if the position value are out
            of limits the return None """
        if x2 >= 0 and x2 < len(self.data) and y2 >= 0 and y2 < len(self.data):
            temp_puz = []
            temp_puz = self.copy(puz)
            temp = temp_puz[x2][y2]
            temp_puz[x2][y2] = temp_puz[x1][y1]
            temp_puz[x1][y1] = temp
            return temp_puz
        else:
            return None
            

    def copy(self, root):
        """ Copy function to create a similar matrix of the given node"""
        temp = []
        for i in root:
            t = []
            for j in i:
                t.append(j)
            temp.append(t)
        return temp    
            
def _find(puz: list, char: str):
    """ Specifically used to find the position of the blank space """
    for i in range(0,len(puz)):
        for j in range(0,len(puz)):
            if puz[i][j] == char:
                return i,j

def _get_f(node: Node, type_: str = "h1"):
    return _get_h(node, type_=type_) + node.level

def _get_h(node: Node, type_: str = "h1"):
    """ Calculates the different between the given puzzles """
    goal = [["_", 1, 2], [3, 4, 5], [6, 7, 8]]
    temp = 0
    for i in range(0,3):
        for j in range(0,3):
            if node.data[i][j] != goal[i][j] and node.data[i][j] != '_':
                if type_ == "h1":
                    temp += 1
                elif type_ == "h2":
                    goal_x, goal_y = _find(goal, node.data[i][j])
                    temp += abs(goal_x - i) + abs(goal_y - j)
    return temp

class Puzzle:
    def __init__(self, size, type_: str = "h1"):
        self.n = size
        self.type_ = type_

    def process(self):
        # initialization
        start = [[7, 2, 4], [5, "_", 6], [8, 3, 1]]
        goal = [["_", 1, 2], [3, 4, 5], [6, 7, 8]]
        start_node = Node(start,0,0)
        start_f = _get_f(start_node)
        node_queue = [
            {
                "node": start_node, 
                "node_name": str(uuid4()), 
                "f": start_f, 
                "prev_move_direction": "", 
                "prev_node_name": ""
            }
        ]
        visited_node = {}

        iter = 0
        max_iter = 10000
        while iter < max_iter:
            iter += 1
            current_node = node_queue.pop(0)
            visited_node[current_node["node_name"]] = current_node
            # print(f"iteration {iter}")
            # for i in current_node["node"].data:
            #     for j in i:
            #         print(j,end=" ")
            #     print("")
            # print(node_queue)

            # Check if the current node is already the same with the goal
            if(_get_h(current_node["node"], self.type_) == 0 or iter == max_iter):
                print(iter)
                print(current_node["node"].data)
                current_node_name = current_node["node_name"]
                break

            # adding new child to infront of    the queue
            node_queue = current_node["node"].generate_child(current_node["prev_move_direction"], current_node["node_name"], h_type=self.type_) + node_queue
            # sort the queue based on f value
            node_queue.sort(key=lambda x: x["f"])
        
        # print the path backward (from target to initial state)
        with open(f"Output_{self.type_}.txt", "w") as text_file:
            text_file.write("target state \n")
            while True:
                current_node = visited_node[current_node_name]
                prev_node_name = current_node["prev_node_name"]
                for i in current_node["node"].data:
                    for j in i:
                        text_file.write(f"{j} ")
                    text_file.write("\n")
                text_file.write(" \n /'\ \n  | \n  | \n \n")

                if visited_node.get(prev_node_name):
                    current_node_name = prev_node_name
                else:
                    text_file.write("initial state \n")
                    break

            

In [ ]:
puz = Puzzle(3, type_="h2")
puz.process()